In [ ]:
'''
Initial code was copied from:
https://github.com/jason71995/Keras-GAN-Library
'''

In [27]:
import tensorflow as tf
import keras.backend as K
from keras.models import Sequential
from keras.layers import Conv2D,GlobalAveragePooling2D,LeakyReLU,Conv2DTranspose,Activation,BatchNormalization
from keras.optimizers import Adam

from keras.layers import Dense
from keras import initializers
from keras.layers.core import Dropout



noise_dim = 5

def build_generator(input_shape):

    generator = Sequential()
    
    generator.add(Dense(64,
                        input_dim = noise_dim))
    generator.add(BatchNormalization())
    generator.add(LeakyReLU(0.2))
    
#     generator.add(Dense(512))
#     generator.add(BatchNormalization())
#     generator.add(LeakyReLU(0.2))
    

    


    
    generator.add(Dense(105, activation='tanh'))
    return generator


def build_discriminator(input_shape):

    discriminator = Sequential()
    discriminator.add(Dense(64,
                            input_dim=105))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
    
    
#     discriminator.add(Dense(512))
#     discriminator.add(LeakyReLU(0.2))
#     discriminator.add(Dropout(0.3))
    

        

    
    discriminator.add(Dense(1, activation='sigmoid'))

    return discriminator

def build_functions(batch_size, noise_size, image_size, generator, discriminator):

    noise = K.random_normal((batch_size,) + noise_size,0.0,1.0,"float32")
    real_image = K.placeholder((batch_size,) + image_size)

    fake_image = generator(noise)

    d_input = K.concatenate([real_image, fake_image], axis=0)
    pred_real, pred_fake = tf.split(discriminator(d_input), num_or_size_splits = 2, axis = 0)

    pred_real = K.clip(pred_real,K.epsilon(),1-K.epsilon())
    pred_fake = K.clip(pred_fake,K.epsilon(),1-K.epsilon())

    d_loss = -(K.mean(K.log(pred_real)) + K.mean(K.log(1-pred_fake)))
    g_loss = -K.mean(K.log(pred_fake))

    # get updates of mean and variance in batch normalization layers
    d_updates = discriminator.get_updates_for([d_input])
    g_updates = generator.get_updates_for([noise])

    d_training_updates = Adam(lr=0.0001, beta_1=0.0, beta_2=0.9).get_updates(d_loss, discriminator.trainable_weights)
    d_train = K.function([real_image, K.learning_phase()], [d_loss],d_updates + d_training_updates)

    g_training_updates = Adam(lr=0.0001, beta_1=0.0, beta_2=0.9).get_updates(g_loss, generator.trainable_weights)
    g_train = K.function([real_image, K.learning_phase()], [g_loss], g_updates + g_training_updates)

    return d_train,g_train

In [3]:
# Loading MHAD data for action1, all persons and all repeatations of each person
from utils.data_loader import data_loader
data_object= data_loader(matlab_action_path='../gan/')
myData, mymin, mymax = data_object.actions_normalised([1], twoD_true_or_threeD_false=False)
myData.shape

(117562, 105)

In [31]:
import keras

#from gan_libs.DCGAN import build_generator, build_discriminator, build_functions
# from gan_libs.LSGAN import build_generator, build_discriminator, build_functions
# from gan_libs.SNGAN import build_generator, build_discriminator, build_functions
# from gan_libs.WGAN_GP import build_generator, build_discriminator, build_functions

from utils.common import set_gpu_config, predict_images
from utils.draw_pose import draw_pose
import numpy as np

#set_gpu_config("0",0.5)

epoch = 25 + 1
image_size = (1,1,105)
noise_size = (1,1,5)
batch_size = 16

x_train = myData
np.random.shuffle(x_train)

generator = build_generator(noise_size)
#print(generator.summary())
discriminator = build_discriminator(image_size)
#print(discriminator.summary())
d_train, g_train = build_functions(batch_size, noise_size, image_size, generator, discriminator)


number_of_all_data = x_train.shape[0]
number_of_batches = int(number_of_all_data/batch_size)
print('Number of Batches passed in each epoch: ',number_of_batches)

import time
start_time = time.time()
is_first_epoch = True

for e in range(epoch):
    index = 0
    for _ in range(number_of_batches):
        real_images = x_train[index:index+batch_size]
        index =+ batch_size
        real_images.shape = (batch_size,1,1,105)
        d_loss, = d_train([real_images, 1])
        g_loss, = g_train([real_images, 1])
     
    
    print ("[{0}/{1}] d_loss: {2:.4}, g_loss: {3:.4}".format(e, epoch, d_loss, g_loss))
    #generating a sample
    image = generator.predict(np.zeros(shape=(1,5)))
    image = np.array(image)
    draw_pose(image.reshape(105),'output',"e{0}".format(e))
    
    if(is_first_epoch):
        elapsed_time = time.time() - start_time 
        print('\n\nTime Taken for single epoch:')
        print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))
        is_first_epoch = False
    
    if e % 25 == 0 and e > 0:
        generator.save_weights("e{0}_generator.h5".format(e))
        discriminator.save_weights("e{0}_discriminator.h5".format(e))
 
#just monitoring:
##########################################################################################
elapsed_time = time.time() - start_time 
print('\n\n\n\nNumber of parameter for the Generator and discriminator respectively:\n')
print(generator.count_params())
print('')
print(discriminator.count_params())
print('\n\nNumber of Epochs and steps for each epoch:\n')
print('epochs: ',epoch, '   batches: ', number_of_batches)

print('\n\nTime Taken:')
print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))
##########################################################################################
#NOTE: previously in each epoch 1000 steps were iterated. in each iteration data was permitated and a batch was chosen. 
#now data is shuffled first and there are number_of_data/batch_size steps to be passed in a single epoch. 
# since 200 epochs and 1000 steps each was used, now 25 epochs and ˜7300 steps is used so the balance is kept.

Number of Batches passed in each epoch:  7347
[0/26] d_loss: 1.151, g_loss: 0.7963


Time Taken for single epoch:
00:00:54
[1/26] d_loss: 1.295, g_loss: 0.7525
[2/26] d_loss: 1.315, g_loss: 0.6759
[3/26] d_loss: 1.268, g_loss: 0.7669
[4/26] d_loss: 1.324, g_loss: 0.713
[5/26] d_loss: 1.344, g_loss: 0.7542
[6/26] d_loss: 1.337, g_loss: 0.7468
[7/26] d_loss: 1.373, g_loss: 0.7338
[8/26] d_loss: 1.306, g_loss: 0.703
[9/26] d_loss: 1.304, g_loss: 0.8106
[10/26] d_loss: 1.325, g_loss: 0.7296
[11/26] d_loss: 1.303, g_loss: 0.7497
[12/26] d_loss: 1.346, g_loss: 0.6764
[13/26] d_loss: 1.262, g_loss: 0.7625
[14/26] d_loss: 1.303, g_loss: 0.7475
[15/26] d_loss: 1.212, g_loss: 0.7231
[16/26] d_loss: 1.239, g_loss: 0.8326
[17/26] d_loss: 1.334, g_loss: 1.014
[18/26] d_loss: 1.315, g_loss: 0.8033
[19/26] d_loss: 1.255, g_loss: 0.9312
[20/26] d_loss: 1.28, g_loss: 0.9483
[21/26] d_loss: 1.275, g_loss: 0.7471
[22/26] d_loss: 1.271, g_loss: 0.8632
[23/26] d_loss: 1.391, g_loss: 1.01
[24/26] d_loss: 1.

In [18]:
    #gpu details:
    ################----------------------------
from six.moves import cStringIO as StringIO
import gpustat

gpustats = gpustat.new_query()
fp = StringIO()
gpustats.print_formatted(
     fp=fp, no_color=False, show_user=False,
     show_cmd=False, show_pid=False, show_power=False, show_fan_speed=False)

result = fp.getvalue()
print('\n\n')
print(result)
    ################----------------------------




cbrc-All-Series  Mon May  6 14:40:03 2019
[0] GeForce GTX 1080 Ti | 53'C,   0 % |  6283 / 11172 MB | cbrc(5961M) gdm(16M) gdm(50M) cbrc(114M) cbrc(136M)



In [ ]:
generator.load_weights("e400_generator.h5".format(e))
discriminator.load_weights("e400_discriminator.h5".format(e))

In [ ]:
for i in range(100):
    x= np.array((i/100,0,0,0,0)).reshape(1,5)
    image = generator.predict(x)
    image = np.array(image)
    draw_pose(image.reshape(105),'output',"e{0}".format(i))